# Step 6: Convert to Monthly Tradable Signal

This notebook:
1. Converts quarterly MT signal to monthly "as-of" signal
2. Forward-fills signal until next call updates it
3. Creates monthly panel for backtesting


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

# Load config
BASE_DIR = Path('/Users/david/Desktop/MATH-GA 2707/Moving Target')
CONFIG_DIR = BASE_DIR / 'configs'
INTERMEDIATE_DIR = BASE_DIR / 'data' / 'intermediate'

with open(CONFIG_DIR / 'base.json', 'r') as f:
    config = json.load(f)

for key in config['data']:
    config['data'][key] = Path(config['data'][key])

# Load quarterly signal
df_signal = pd.read_parquet(config['data']['firm_quarter_signal'])
df_signal['call_date'] = pd.to_datetime(df_signal['call_date'])

print(f"Loaded {len(df_signal)} quarterly signals")
print(f"Date range: {df_signal['call_date'].min()} to {df_signal['call_date'].max()}")


Loaded 53395 quarterly signals
Date range: 2009-05-29 00:00:00 to 2025-04-15 00:00:00


In [2]:
# Create monthly panel
# Effective date = first trading day of next month after call
df_signal['effective_date'] = (df_signal['call_date'] + pd.offsets.MonthBegin(1)).dt.to_period('M').dt.to_timestamp()

# Generate all months in range
start_month = df_signal['effective_date'].min().to_period('M')
end_month = df_signal['effective_date'].max().to_period('M') + 12  # Extend 12 months for forward-fill

all_months = pd.period_range(start_month, end_month, freq='M')
all_firms = df_signal['firm_id'].unique()

# Create full monthly panel
monthly_panel = []
for firm_id in all_firms:
    firm_signals = df_signal[df_signal['firm_id'] == firm_id].sort_values('effective_date')
    
    for month in all_months:
        month_end = month.to_timestamp() + pd.offsets.MonthEnd(0)
        
        # Find latest signal with effective_date <= month_end
        valid_signals = firm_signals[firm_signals['effective_date'] <= month_end]
        
        if len(valid_signals) > 0:
            latest_signal = valid_signals.iloc[-1]
            monthly_panel.append({
                'firm_id': firm_id,
                'ticker': latest_signal['ticker'],
                'month_end': month_end,
                'MT_asof': latest_signal['MT'],
                'MT_persistent_asof': latest_signal['MT_persistent'],
                'n_targets_lag4': latest_signal['n_targets_lag4'],
                'complexity': latest_signal['complexity'],
                'last_call_date': latest_signal['call_date']
            })

df_monthly = pd.DataFrame(monthly_panel)
print(f"Created monthly panel with {len(df_monthly)} firm-month observations")


Created monthly panel with 218052 firm-month observations


In [3]:
# Statistics
print("Monthly Signal Statistics:")
print(f"  Unique firms: {df_monthly['firm_id'].nunique()}")
print(f"  Month range: {df_monthly['month_end'].min()} to {df_monthly['month_end'].max()}")
print(f"  Valid MT signals: {df_monthly['MT_asof'].notna().sum()}")
print(f"  Average MT: {df_monthly['MT_asof'].mean():.4f}")

# Save monthly signal
output_file = config['data']['monthly_signal']
df_monthly.to_parquet(output_file, index=False, engine='pyarrow')
print(f"\nSaved monthly signal to: {output_file}")


Monthly Signal Statistics:
  Unique firms: 1472
  Month range: 2009-06-30 00:00:00 to 2026-05-31 00:00:00
  Valid MT signals: 92144
  Average MT: 0.8643

Saved monthly signal to: /Users/david/Desktop/MATH-GA 2707/Moving Target/data/intermediate/monthly_signal.parquet
